In [39]:
import pandas as pd
import numpy as np
import mysql.connector

In [40]:
top_250 = pd.read_csv('top_250.csv')

In [41]:
top_250['parental_guide'].replace(['blank' , 'Not Rated' , 'null'] , 'Unrated' , inplace=True)

In [42]:
top_250['gross_us_canada']=top_250['gross_us_canada'].str.replace('[,|$]', '' ,regex=True)

In [43]:
top_250.head()

,title,runtime,parental_guide,genre,directors,stars,writers,person_id,gross_us_canada,film_id
0,The Shawshank Redemption,2h 22m,R,['Drama'],['Frank Darabont'],"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton']","['Stephen King', 'Frank Darabont']","{'directors': ['0001104'], 'stars': ['0000209'...",28767189,111161
1,The Godfather,2h 55m,R,"['Crime', 'Drama']",['Francis Ford Coppola'],"['Marlon Brando', 'Al Pacino', 'James Caan']","['Mario Puzo', 'Francis Ford Coppola']","{'directors': ['0000338'], 'stars': ['0000008'...",136381073,68646
2,The Dark Knight,2h 32m,PG-13,"['Action', 'Crime', 'Drama']",['Christopher Nolan'],"['Christian Bale', 'Heath Ledger', 'Aaron Eckh...","['Jonathan Nolan', 'Christopher Nolan', 'David...","{'directors': ['0634240'], 'stars': ['0000288'...",534987076,468569
3,The Godfather Part II,3h 22m,R,"['Crime', 'Drama']",['Francis Ford Coppola'],"['Al Pacino', 'Robert De Niro', 'Robert Duvall']","['Francis Ford Coppola', 'Mario Puzo']","{'directors': ['0000338'], 'stars': ['0000199'...",47834595,71562
4,12 Angry Men,1h 36m,Approved,"['Crime', 'Drama']",['Sidney Lumet'],"['Henry Fonda', 'Lee J. Cobb', 'Martin Balsam']",['Reginald Rose'],"{'directors': ['0001486'], 'stars': ['0000020'...",NaN,50083


In [44]:
database = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'Sh@0831@Sh'
)

In [ ]:
mycursor = database.cursor()

In [45]:

mycursor.execute('create database IMDB_TOP_250')


In [46]:
mycursor.execute('USE IMDB_TOP_250')

In [48]:
mycursor.execute('create table  movie (id VARCHAR(8) PRIMARY KEY , \
                 title VARCHAR(128) NOT NULL,\
                 year INT,\
                 runtime INT,\
                 parental_guide VARCHAR(8),\
                 gross_us_canada INT)')


In [49]:
mycursor.execute('create table person (id varchar(8) PRIMARY KEY ,\
                 name varchar(32))')

In [51]:
mycursor.execute('CREATE TABLE cast (id INT AUTO_INCREMENT PRIMARY KEY,\
    movie_id VARCHAR(8),\
    person_id VARCHAR(8),\
    FOREIGN KEY (movie_id) REFERENCES movie(id),\
    FOREIGN KEY (person_id) REFERENCES person(id)\
) AUTO_INCREMENT = 1')